Import Packages (Same as before)

In [1]:
###################### STEP 1 ###################### 
#import packages
import requests
import time
import os
import pandas as pd
import numpy as np
import random
import re
import regex

# !pip3 install -U spacy
import spacy

from time import sleep
from os import path
from pandas import DataFrame
from bs4 import BeautifulSoup
from random import randint

# import selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.common.exceptions import NoSuchElementException
# from selenium.common.exceptions import ElementClickInterceptedException
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains

# set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# specify file path of chromedriver
# s = Service('/home/himi/Documents/RCC/rent_scraper/chromedriver')

snapshot_links, name, owner, address, price_range, bed_range, amenities, contact, property_link = [], [], [], [], [], [], [], [], []

In [15]:
contacts_re = r'\(?[0-9]{3}\)?\/?\.?-?\s?[0-9]{3}-?\.?-?\s?[0-9]{4}'
address_re = r'[\s\n]*[1-9][0-9]{,4} (?:[A-Z][a-zA-Z]*,?\s*){,5} Chicago,?[\w\s,]{,3}\s*[0-9]{,6}?'
bed_range_re = r'Studio\s*-\s*[1-9]\s*[a-zA-Z]+|[1-9]\s*-\s*[1-9]\s*[a-zA-Z]+|\s*Studio$|\s*[1-9]\s*[a-zA-Z]+|No Availability'
price_re = r'\$[1-9]?[0-9]{,3},?[0-9]{,3}\s*-\s*\$?[1-9]?[0-9]{,3},?[0-9]{,3}|\$[1-9]?[0-9]{,3},?[0-9]{,3}$|Call for Rent'

Get snapshot links (Same as before)

In [8]:
###################### STEP 2 ###################### 

# open chrome
browser = webdriver.Chrome(service=s)

# define action
action = ActionChains(browser)

# list of years to loop over
years = ["2015"]


# scrape snapshot links
for y in years:
    
    # open link
    print("".join(["start scraping blue snapshots from ", y]))
    yearpage = "https://web.archive.org/web/" + y + "0000000000*/https://www.apartments.com/chicago-il/"
    browser.get(yearpage)
    time.sleep(10)
    
    # find all blue snapshots
    blues = browser.find_elements("xpath","//*[@class='measure s2xx']")
    time.sleep(5)
    
    # hover over blue snapshot and fetch snapshot link elements
    for blue in blues:
        day = blue.find_element("xpath","../..")
        time.sleep(2)
        action.move_to_element(day).perform()
        time.sleep(2)
        popup = browser.find_element("xpath","//*[@class='popup-of-day']")
        time.sleep(2)
        action.move_to_element(popup).perform()
        time.sleep(2)
        hrefs = browser.find_elements("xpath","//*[@class='s2xx snapshot-link']")
        
        # save each link and append to list
        for href in hrefs: 
            link = href.get_attribute("href")
            time.sleep(2)
            snapshot_links.append(link)

    # done with year
    print("".join(["* done with ", y]))
    
# close chrome            
browser.quit()

start scraping blue snapshots from 2015
* done with 2015


Function to get all the contents from a particular webpage

In [2]:
def get_contents(soup, content_text):
  try:
    parents_blacklist=['[document]','html','head',
                       'style','script','body',
                       'section','tr',
                       'td','label','ul','header',
                       'aside']
    content=''
    text=soup.find_all(text=True)

    
    for t in text:
        if t.parent.name not in parents_blacklist and len(t) > 5:
            content=content+t+' '
    content_text.append(content)
  except Exception:
    content_text.append('')
    pass



In [31]:
def get_regex_data(webpage):
    page=requests.get(webpage)
    soup=BeautifulSoup(page.text,'html.parser')
    content_text = []
    get_contents(soup, content_text)
#     print(content_text[0])
    contacts = re.findall(r'\(?[0-9]{3}\)?\/?\.?-?\s?[0-9]{3}-?\.?-?\s?[0-9]{4}', content_text[0])
    address = regex.findall(r'[\s\n]*[1-9][0-9]{,4} (?:[A-Z][a-zA-Z]*,?\s*){,5} Chicago,?[\w\s,]{,3}\s*[0-9]{,6}?', content_text[0])
    address = [addi.strip() for addi in address]
    bed_range = re.findall(r'Studio\s*-\s*[1-9]\s*[a-zA-Z]+|[1-9]\s*-\s*[1-9]\s*[a-zA-Z]+|\s*Studio$|\s*[1-9]\s*[a-zA-Z]+$|\s*No Availability\s*', content_text[0])
    prices = re.findall(r'\$[1-9]?[0-9]{,3},?[0-9]{,3}\s*-\s*\$?[1-9]?[0-9]{,3},?[0-9]{,3}|\$[1-9]?[0-9]{,3},?[0-9]{,3}$|Call for Rent', content_text[0])
    print("\nContacts are:", contacts)
    print("\nAddresses are:", address)
    print("\nPrices are:" , prices)
    print("\nBeds are:" , bed_range)
    print("\nLengths of Filtered Lists are", len(contacts), len(address), len(prices), len(bed_range))

Function to get phone numbers, address and prices using regular expressions

In [58]:
def consolidate(output):
    label_list = [{}]
    output_final = []
    property_info = {'Address': None, 'Price': None, 'Beds': None, 'Contact' : None}
    for i, out in enumerate(output):
        if re.match(address, out):
            if property_info['Address'] != None:
                output_final.append(property_info)
                property_info = {'Address': None, 'Price': None, 'Beds': None, 'Contact' : None}
                property_info['Address'] = out
            else:
                property_info['Address'] = out
        elif re.match(prices, out):
            if property_info['Price'] != None:
                output_final.append(property_info)
                property_info = {'Address': None, 'Price': None, 'Beds': None, 'Contact' : None}
                property_info['Price'] = out
            else:
                property_info['Price'] = out
        elif re.match(bed_range,out):
            if property_info['Beds'] != None:
                output_final.append(property_info)
                property_info = {'Address': None, 'Price': None, 'Beds': None, 'Contact' : None}
                property_info['Beds'] = out
            else:
                property_info['Beds'] = out
        elif re.match(contacts,out):
            if property_info['Contact'] != None:
                output_final.append(property_info)
                property_info = {'Address': None, 'Price': None, 'Beds': None, 'Contact' : None}
                property_info['Contact'] = out
            else:
                property_info['Contact'] = out
#     print(output_final)
    return output_final

In [12]:
def get_regex_data_primary(webpage):
    page=requests.get(webpage)
    soup=BeautifulSoup(page.text,'html.parser')
    content_text = []
    get_contents(soup, content_text)
    contacts = r'\(?[0-9]{3}\)?\/?\.?-?\s?[0-9]{3}-?\.?-?\s?[0-9]{4}'
    address = r'[\s\n]*[1-9][0-9]{,4} (?:[A-Z][a-zA-Z]*,?\s*){,5} Chicago,?[\w\s,]{,3}\s*[0-9]{,6}?'
#     address = [addi.strip() for addi in address]
    bed_range = r'Studio\s*-\s*[1-9]\s*[a-zA-Z]+|[1-9]\s*-\s*[1-9]\s*[a-zA-Z]+|\s*Studio$|\s*[1-9]\s*[a-zA-Z]+'
    prices = r'\$[1-9]?[0-9]{,3},?[0-9]{,3}\s*-\s*\$?[1-9]?[0-9]{,3},?[0-9]{,3}|\$[1-9]?[0-9]{,3},?[0-9]{,3}$|Call for Rent'
    total = r'\(?[0-9]{3}\)?\/?\.?-?\s?[0-9]{3}-?\.?-?\s?[0-9]{4}|[\s\n]*[1-9][0-9]{,4} (?:[A-Z][a-zA-Z]*,?\s*){,5} Chicago,?[\w\s,]{,3}\s*[0-9]{,6}?|Studio\s*-\s*[1-9]\s*[a-zA-Z]+|[1-9]\s*-\s*[1-9]\s*[a-zA-Z]+|\s*Studio$|\s*[1-9]\s*[a-zA-Z]+,|\$[1-9]?[0-9]{,3},?[0-9]{,3}\s*-\s*\$?[1-9]?[0-9]{,3},?[0-9]{,3}|\$[1-9]?[0-9]{,3},?[0-9]{,3}$|Call for Rent'
    output = re.findall(total,content_text[0])
    output = [out.strip() for out in output]
    return output

In [32]:
# link="https://web.archive.org/web/20150217144133/https://www.apartments.com/chicago-il/"
# link = "https://web.archive.org/web/20220819053652/https://www.apartments.com/chicago-il/"
link = "https://web.archive.org/web/20160301213254/https://www.apartments.com/chicago-il/2/"

print("For test link")
get_regex_data(link)

For test link

Contacts are: ['312-462-0403', '844-737-7559', '312-219-9516', '844-250-5735', '844-741-2982', '844-837-3913', '844-294-8524', '844-406-5855', '844-841-6325', '844-368-4250', '844-525-6919', '844-811-0188', '773-675-1758', '312-646-4953', '866-661-2651', '773-570-3150', '877-331-6062', '708-546-4173', '312-789-5353', '844-233-2845', '888-449-2602', '888-380-0393', '773-358-4744', '877-834-2986', '773-649-4066']

Addresses are: ['1350 N Lake Shore Dr Chicago', '1212 W Madison St Chicago', '1 W Superior St Chicago', '37 W Van Buren St Chicago', '910 W Huron St Chicago', '707 N Wells St Chicago', '730 W Couch Pl Chicago', '1225 N Wells St Chicago', '365 N Halsted St Chicago', '625 W Division St Chicago', '343 W Wolf Point Plz Chicago', '14 W Elm St Chicago', '355 E Ohio St Chicago', '180 N Jefferson St Chicago', '448 E Ontario St Chicago', '188 W Randolph St Chicago', '111 W Wacker Dr Chicago', '150 W Roosevelt Rd Chicago', '73 E Lake St Chicago', '215 E Chestnut St Chicago

In [7]:
output = get_regex_data_primary(link)
data = consolidate(output)
df = pd.DataFrame(data)
print(df)

[{'Address': '1350 N Lake Shore Dr Chicago', 'Price': '$1,272 - $2,994', 'Beds': 'Studio - 2 Br', 'Contact': '312-462-0403'}, {'Address': '1212 W Madison St Chicago', 'Price': '$1,875 - $3,250', 'Beds': 'Studio - 2 Br', 'Contact': '844-737-7559'}, {'Address': '1 W Superior St Chicago', 'Price': '$1,636 - $4,673', 'Beds': 'Studio - 2 Br', 'Contact': '312-219-9516'}, {'Address': '37 W Van Buren St Chicago', 'Price': None, 'Beds': 'Studio - 4 Br', 'Contact': '844-250-5735'}, {'Address': '910 W Huron St Chicago', 'Price': '$1,925 - $3,410', 'Beds': 'Studio - 4 Br', 'Contact': '844-741-2982'}, {'Address': '707 N Wells St Chicago', 'Price': '$2,740 - $5,621', 'Beds': '1-3 Br', 'Contact': '844-837-3913'}, {'Address': '730 W Couch Pl Chicago', 'Price': '$1,750 - $5,250', 'Beds': 'Studio - 3 Br', 'Contact': '844-294-8524'}, {'Address': '1225 N Wells St Chicago', 'Price': '$1,105 - $2,530', 'Beds': 'Studio - 3 Br', 'Contact': '844-406-5855'}, {'Address': '365 N Halsted St Chicago', 'Price': '$1,

In [57]:
info_2016 = pd.read_csv("../training_data/apartments_com_prices_2016.csv", delimiter= ",")
info_2016 = info_2016.drop(['Unnamed: 0'], axis = 1)
links = info_2016['Link']
unique_links = list(set(links))
df_final = []
for link in unique_links[1:10]:
    output = get_regex_data_primary(link)
    data = consolidate(output)
    df = pd.DataFrame(data)
    df_final.append(df)

merged = pd.concat(df_final)
print(merged)
    

                             Address             Price           Beds  \
0         601 W Jackson Blvd Chicago   $1,466 - $4,840  Studio - 2 Br   
1              420 E Ohio St Chicago   $1,605 - $4,600  Studio - 3 Br   
2        500 N Milwaukee Ave Chicago   $1,580 - $5,500  Studio - 3 Br   
3         840 W Blackhawk St Chicago   $2,085 - $4,537  Studio - 2 Br   
4            1 W Superior St Chicago   $1,676 - $4,131  Studio - 2 Br   
5          200 E Illinois St Chicago  $2,268 - $12,878  Studio - 3 Br   
6          625 W Division St Chicago   $1,875 - $3,650  Studio - 2 Br   
7             707 N Wells St Chicago   $2,707 - $5,521         1-3 Br   
8       343 W Wolf Point Plz Chicago  $1,908 - $10,900  Studio - 3 Br   
9          225 N Columbus Dr Chicago   $2,028 - $5,392  Studio - 2 Br   
10      400 E South Water St Chicago   $1,910 - $4,564  Studio - 2 Br   
11      360 E South Water St Chicago   $1,696 - $5,480  Studio - 2 Br   
12           345 E Wacker Dr Chicago   $2,151 - $4,

In [60]:
print(merged.isnull().sum())

Address     2
Price      17
Beds        9
Contact     0
dtype: int64


In [61]:
merged.to_csv("info_2016_extracted.csv")